In [1]:
# import the necessary packages
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from HDF5file import HDF5DatasetWriter
from imutils import paths
import numpy as np
import progressbar
import argparse
import random
import os

Using TensorFlow backend.


In [4]:
dataset = "Rotated/" #add your generated datset path here
output = "HDF5file"
batch_size = 32
buffer_size = 1000

In [5]:
# grab the list of images that we'll be describing then randomly
# shuffle them to allow for easy training and testing splits via
# array slicing during training time
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset))
random.shuffle(imagePaths)

[INFO] loading images...


In [6]:
# extract the class labels from the image paths then encode the
# labels
labels = [p.split(os.path.sep)[-2] for p in imagePaths]
le = LabelEncoder()
labels = le.fit_transform(labels)

In [7]:
# load the VGG16 network
print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)

[INFO] loading network...


In [9]:
# initialize the HDF5 dataset writer, then store the class label
# names in the dataset
dataset = HDF5DatasetWriter((len(imagePaths), 512 * 7 * 7),
	output, dataKey="features", bufSize=buffer_size)
dataset.storeClassLabels(le.classes_)

In [10]:
# initialize the progress bar
widgets = ["Extracting Features: ", progressbar.Percentage(), " ",
	progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(imagePaths),
	widgets=widgets).start()

In [11]:
bs = batch_size
# loop over the images in patches
for i in np.arange(0, len(imagePaths), bs):
	# extract the batch of images and labels, then initialize the
	# list of actual images that will be passed through the network
	# for feature extraction
	batchPaths = imagePaths[i:i + bs]
	batchLabels = labels[i:i + bs]
	batchImages = []

	# loop over the images and labels in the current batch
	for (j, imagePath) in enumerate(batchPaths):
		# load the input image using the Keras helper utility
		# while ensuring the image is resized to 224x224 pixels
		image = load_img(imagePath, target_size=(224, 224))
		image = img_to_array(image)

		# preprocess the image by (1) expanding the dimensions and
		# (2) subtracting the mean RGB pixel intensity from the
		# ImageNet dataset
		image = np.expand_dims(image, axis=0)
		image = imagenet_utils.preprocess_input(image)

		# add the image to the batch
		batchImages.append(image)

	# pass the images through the network and use the outputs as
	# our actual features
	batchImages = np.vstack(batchImages)
	features = model.predict(batchImages, batch_size=bs)

	# reshape the features so that each image is represented by
	# a flattened feature vector of the `MaxPooling2D` outputs
	features = features.reshape((features.shape[0], 512 * 7 * 7))

	# add the features and labels to our HDF5 dataset
	dataset.add(features, batchLabels)
	pbar.update(i)

# close the dataset
dataset.close()
pbar.finish()

Extracting Features: 100% |#####################################| Time: 1:15:05
